In [1]:
#Read dataset in Spark
df = sqlContext.read.load("dbfs:/databricks-datasets/bikeSharing/data-001/day.csv", 
                          format='com.databricks.spark.csv', 
                          header='true', 
                          inferSchema='true')
#This is databrick dataset of bike sharing, have done project with given dataset though

In [2]:
#2. Get summary of data and variable types
df.printSchema()

root
-- instant: integer (nullable = true)
-- dteday: timestamp (nullable = true)
-- season: integer (nullable = true)
-- yr: integer (nullable = true)
-- mnth: integer (nullable = true)
-- holiday: integer (nullable = true)
-- weekday: integer (nullable = true)
-- workingday: integer (nullable = true)
-- weathersit: integer (nullable = true)
-- temp: double (nullable = true)
-- atemp: double (nullable = true)
-- hum: double (nullable = true)
-- windspeed: double (nullable = true)
-- casual: integer (nullable = true)
-- registered: integer (nullable = true)
-- cnt: integer (nullable = true)

In [3]:
df.show(5)

+-------+-------------------+------+---+----+-------+-------+----------+----------+--------+--------+--------+---------+------+----------+----+
instant| dteday|season| yr|mnth|holiday|weekday|workingday|weathersit| temp| atemp| hum|windspeed|casual|registered| cnt|
+-------+-------------------+------+---+----+-------+-------+----------+----------+--------+--------+--------+---------+------+----------+----+
 1|2011-01-01 00:00:00| 1| 0| 1| 0| 6| 0| 2|0.344167|0.363625|0.805833| 0.160446| 331| 654| 985|
 2|2011-01-02 00:00:00| 1| 0| 1| 0| 0| 0| 2|0.363478|0.353739|0.696087| 0.248539| 131| 670| 801|
 3|2011-01-03 00:00:00| 1| 0| 1| 0| 1| 1| 1|0.196364|0.189405|0.437273| 0.248309| 120| 1229|1349|
 4|2011-01-04 00:00:00| 1| 0| 1| 0| 2| 1| 1| 0.2|0.212122|0.590435| 0.160296| 108| 1454|1562|
 5|2011-01-05 00:00:00| 1| 0| 1| 0| 3| 1| 1|0.226957| 0.22927|0.436957| 0.1869| 82| 1518|1600|
+-------+-------------------+------+---+----+-------+-------+----------+----------+--------+--------+--------+---------+------+----------+----+
only showing top 5 rows

In [4]:
#Given Train file from which data frame is generated 
bs_df = spark.sql("select * from bike_sharing_train_csv")
bs_df.show(5)

+----------------+------+-------+----------+-------+----+------+--------+---------+------+----------+-----+
 datetime|season|holiday|workingday|weather|temp| atemp|humidity|windspeed|casual|registered|count|
+----------------+------+-------+----------+-------+----+------+--------+---------+------+----------+-----+
01-01-2011 00:00| 1| 0| 0| 1|9.84|14.395| 81| 0.0| 3| 13| 16|
01-01-2011 01:00| 1| 0| 0| 1|9.02|13.635| 80| 0.0| 8| 32| 40|
01-01-2011 02:00| 1| 0| 0| 1|9.02|13.635| 80| 0.0| 5| 27| 32|
01-01-2011 03:00| 1| 0| 0| 1|9.84|14.395| 75| 0.0| 3| 10| 13|
01-01-2011 04:00| 1| 0| 0| 1|9.84|14.395| 75| 0.0| 0| 1| 1|
+----------------+------+-------+----------+-------+----+------+--------+---------+------+----------+-----+
only showing top 5 rows

In [5]:
bs_df.printSchema()

root
-- datetime: string (nullable = true)
-- season: integer (nullable = true)
-- holiday: integer (nullable = true)
-- workingday: integer (nullable = true)
-- weather: integer (nullable = true)
-- temp: double (nullable = true)
-- atemp: double (nullable = true)
-- humidity: integer (nullable = true)
-- windspeed: double (nullable = true)
-- casual: integer (nullable = true)
-- registered: integer (nullable = true)
-- count: integer (nullable = true)

In [6]:
bs_df.describe().show()

+-------+----------------+------------------+-------------------+------------------+------------------+------------------+-----------------+------------------+------------------+-----------------+------------------+------------------+
summary| datetime| season| holiday| workingday| weather| temp| atemp| humidity| windspeed| casual| registered| count|
+-------+----------------+------------------+-------------------+------------------+------------------+------------------+-----------------+------------------+------------------+-----------------+------------------+------------------+
 count| 10886| 10886| 10886| 10886| 10886| 10886| 10886| 10886| 10886| 10886| 10886| 10886|
 mean| null|2.5066139996325556|0.02856880396839978|0.6808745177291935| 1.418427337865148|20.230859819952173|23.65508405291192| 61.88645967297446|12.799395406945093|36.02195480433584| 155.5521771082124|191.57413191254824|
 stddev| null|1.1161743093443237|0.16659885062470944|0.4661591687997361|0.6338385858190968| 7.791589843987573| 8.47460062648494|19.245033277394704| 8.16453732683871|49.96047657264955|151.03903308192452|181.14445383028493|
 min|01-01-2011 00:00| 1| 0| 0| 1| 0.82| 0.76| 0| 0.0| 0| 0| 1|
 max|19-12-2012 23:00| 4| 1| 1| 4| 41.0| 45.455| 100| 56.9969| 367| 886| 977|
+-------+----------------+------------------+-------------------+------------------+------------------+------------------+-----------------+------------------+------------------+-----------------+------------------+------------------+

In [7]:
bs_df.explain()

== Physical Plan ==
*(1) FileScan csv default.bike_sharing_train_csv[datetime#153,season#154,holiday#155,workingday#156,weather#157,temp#158,atemp#159,humidity#160,windspeed#161,casual#162,registered#163,count#164] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[dbfs:/FileStore/tables/train.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<datetime:string,season:int,holiday:int,workingday:int,weather:int,temp:double,atemp:double...

In [8]:
#Check for any missing value in dataset and treat it
print(bs_df.count())
df_no_null = bs_df.na.drop()
print(df_no_null.count())

10886
10886

In [9]:
#Check what are the distinct seasons present to explode them
bs_df.select('season').distinct().show()

+------+
season|
+------+
 1|
 3|
 4|
 2|
+------+

In [10]:
#user defined function to help creat new columns
def valueToCategory(value, encoding_index):
   if(value == encoding_index):
      return 1
   else:
    return 0

In [11]:
#Explode season column into separate columns such as season_<val> and drop season
from pyspark.sql.functions import udf
from pyspark.sql.functions import lit
from pyspark.sql.types import *
from pyspark.sql.functions import col
udfValueToCategory = udf(valueToCategory, IntegerType())
bs_df_encoded = (bs_df.withColumn("season_1", udfValueToCategory(col('season'),lit(1)))
                     .withColumn("season_2", udfValueToCategory(col('season'),lit(2)))
                     .withColumn("season_3", udfValueToCategory(col('season'),lit(3)))
                     .withColumn("season_4", udfValueToCategory(col('season'),lit(4))))
bs_df_encoded = bs_df_encoded.drop('season')

In [12]:
bs_df_encoded.show()

+----------------+-------+----------+-------+-----+------+--------+---------+------+----------+-----+--------+--------+--------+--------+
 datetime|holiday|workingday|weather| temp| atemp|humidity|windspeed|casual|registered|count|season_1|season_2|season_3|season_4|
+----------------+-------+----------+-------+-----+------+--------+---------+------+----------+-----+--------+--------+--------+--------+
01-01-2011 00:00| 0| 0| 1| 9.84|14.395| 81| 0.0| 3| 13| 16| 1| 0| 0| 0|
01-01-2011 01:00| 0| 0| 1| 9.02|13.635| 80| 0.0| 8| 32| 40| 1| 0| 0| 0|
01-01-2011 02:00| 0| 0| 1| 9.02|13.635| 80| 0.0| 5| 27| 32| 1| 0| 0| 0|
01-01-2011 03:00| 0| 0| 1| 9.84|14.395| 75| 0.0| 3| 10| 13| 1| 0| 0| 0|
01-01-2011 04:00| 0| 0| 1| 9.84|14.395| 75| 0.0| 0| 1| 1| 1| 0| 0| 0|
01-01-2011 05:00| 0| 0| 2| 9.84| 12.88| 75| 6.0032| 0| 1| 1| 1| 0| 0| 0|
01-01-2011 06:00| 0| 0| 1| 9.02|13.635| 80| 0.0| 2| 0| 2| 1| 0| 0| 0|
01-01-2011 07:00| 0| 0| 1| 8.2| 12.88| 86| 0.0| 1| 2| 3| 1| 0| 0| 0|
01-01-2011 08:00| 0| 0| 1| 9.84|14.395| 75| 0.0| 1| 7| 8| 1| 0| 0| 0|
01-01-2011 09:00| 0| 0| 1|13.12|17.425| 76| 0.0| 8| 6| 14| 1| 0| 0| 0|
01-01-2011 10:00| 0| 0| 1|15.58|19.695| 76| 16.9979| 12| 24| 36| 1| 0| 0| 0|
01-01-2011 11:00| 0| 0| 1|14.76|16.665| 81| 19.0012| 26| 30| 56| 1| 0| 0| 0|
01-01-2011 12:00| 0| 0| 1|17.22| 21.21| 77| 19.0012| 29| 55| 84| 1| 0| 0| 0|
01-01-2011 13:00| 0| 0| 2|18.86|22.725| 72| 19.9995| 47| 47| 94| 1| 0| 0| 0|
01-01-2011 14:00| 0| 0| 2|18.86|22.725| 72| 19.0012| 35| 71| 106| 1| 0| 0| 0|
01-01-2011 15:00| 0| 0| 2|18.04| 21.97| 77| 19.9995| 40| 70| 110| 1| 0| 0| 0|
01-01-2011 16:00| 0| 0| 2|17.22| 21.21| 82| 19.9995| 41| 52| 93| 1| 0| 0| 0|
01-01-2011 17:00| 0| 0| 2|18.04| 21.97| 82| 19.0012| 15| 52| 67| 1| 0| 0| 0|
01-01-2011 18:00| 0| 0| 3|17.22| 21.21| 88| 16.9979| 9| 26| 35| 1| 0| 0| 0|
01-01-2011 19:00| 0| 0| 3|17.22| 21.21| 88| 16.9979| 6| 31| 37| 1| 0| 0| 0|
+----------------+-------+----------+-------+-----+------+--------+---------+------+----------+-----+--------+--------+--------+--------+
only showing top 20 rows

In [13]:
#Execute the same for weather as weather_<val> and drop weather
bs_df.select('weather').distinct().show()

+-------+
weather|
+-------+
 1|
 3|
 4|
 2|
+-------+

In [14]:
bs_df_encoded = (bs_df_encoded.withColumn("weather_1", udfValueToCategory(col('weather'),lit(1)))
                     .withColumn("weather_2", udfValueToCategory(col('weather'),lit(2)))
                     .withColumn("weather_3", udfValueToCategory(col('weather'),lit(3)))
                     .withColumn("weather_4", udfValueToCategory(col('weather'),lit(4))))
bs_df_encoded = bs_df_encoded.drop('weather')

In [15]:
bs_df_encoded.show()

+----------------+-------+----------+-----+------+--------+---------+------+----------+-----+--------+--------+--------+--------+---------+---------+---------+---------+
 datetime|holiday|workingday| temp| atemp|humidity|windspeed|casual|registered|count|season_1|season_2|season_3|season_4|weather_1|weather_2|weather_3|weather_4|
+----------------+-------+----------+-----+------+--------+---------+------+----------+-----+--------+--------+--------+--------+---------+---------+---------+---------+
01-01-2011 00:00| 0| 0| 9.84|14.395| 81| 0.0| 3| 13| 16| 1| 0| 0| 0| 1| 0| 0| 0|
01-01-2011 01:00| 0| 0| 9.02|13.635| 80| 0.0| 8| 32| 40| 1| 0| 0| 0| 1| 0| 0| 0|
01-01-2011 02:00| 0| 0| 9.02|13.635| 80| 0.0| 5| 27| 32| 1| 0| 0| 0| 1| 0| 0| 0|
01-01-2011 03:00| 0| 0| 9.84|14.395| 75| 0.0| 3| 10| 13| 1| 0| 0| 0| 1| 0| 0| 0|
01-01-2011 04:00| 0| 0| 9.84|14.395| 75| 0.0| 0| 1| 1| 1| 0| 0| 0| 1| 0| 0| 0|
01-01-2011 05:00| 0| 0| 9.84| 12.88| 75| 6.0032| 0| 1| 1| 1| 0| 0| 0| 0| 1| 0| 0|
01-01-2011 06:00| 0| 0| 9.02|13.635| 80| 0.0| 2| 0| 2| 1| 0| 0| 0| 1| 0| 0| 0|
01-01-2011 07:00| 0| 0| 8.2| 12.88| 86| 0.0| 1| 2| 3| 1| 0| 0| 0| 1| 0| 0| 0|
01-01-2011 08:00| 0| 0| 9.84|14.395| 75| 0.0| 1| 7| 8| 1| 0| 0| 0| 1| 0| 0| 0|
01-01-2011 09:00| 0| 0|13.12|17.425| 76| 0.0| 8| 6| 14| 1| 0| 0| 0| 1| 0| 0| 0|
01-01-2011 10:00| 0| 0|15.58|19.695| 76| 16.9979| 12| 24| 36| 1| 0| 0| 0| 1| 0| 0| 0|
01-01-2011 11:00| 0| 0|14.76|16.665| 81| 19.0012| 26| 30| 56| 1| 0| 0| 0| 1| 0| 0| 0|
01-01-2011 12:00| 0| 0|17.22| 21.21| 77| 19.0012| 29| 55| 84| 1| 0| 0| 0| 1| 0| 0| 0|
01-01-2011 13:00| 0| 0|18.86|22.725| 72| 19.9995| 47| 47| 94| 1| 0| 0| 0| 0| 1| 0| 0|
01-01-2011 14:00| 0| 0|18.86|22.725| 72| 19.0012| 35| 71| 106| 1| 0| 0| 0| 0| 1| 0| 0|
01-01-2011 15:00| 0| 0|18.04| 21.97| 77| 19.9995| 40| 70| 110| 1| 0| 0| 0| 0| 1| 0| 0|
01-01-2011 16:00| 0| 0|17.22| 21.21| 82| 19.9995| 41| 52| 93| 1| 0| 0| 0| 0| 1| 0| 0|
01-01-2011 17:00| 0| 0|18.04| 21.97| 82| 19.0012| 15| 52| 67| 1| 0| 0| 0| 0| 1| 0| 0|
01-01-2011 18:00| 0| 0|17.22| 21.21| 88| 16.9979| 9| 26| 35| 1| 0| 0| 0| 0| 0| 1| 0|
01-01-2011 19:00| 0| 0|17.22| 21.21| 88| 16.9979| 6| 31| 37| 1| 0| 0| 0| 0| 0| 1| 0|
+----------------+-------+----------+-----+------+--------+---------+------+----------+-----+--------+--------+--------+--------+---------+---------+---------+---------+
only showing top 20 rows

In [16]:
# Split datetime into meaningful columns such as hour,day,month,year,etc
from pyspark.sql.functions import split
from pyspark.sql.functions import *
from pyspark.sql.types import *
bs_df_encoded = bs_df_encoded.withColumn('hour',  split(split(bs_df_encoded['datetime'], ' ')[1], ':')[0].cast('int'))
bs_df_encoded = bs_df_encoded.withColumn('month', split(split(bs_df_encoded['datetime'], ' ')[0], '-')[0].cast('int'))
bs_df_encoded = bs_df_encoded.withColumn('day', split(split(bs_df_encoded['datetime'], ' ')[0], '-')[1].cast('int'))
bs_df_encoded = bs_df_encoded.withColumn('year', split(split(bs_df_encoded['datetime'], ' ')[0], '-')[2].cast('int'))

In [17]:
display(bs_df_encoded)

datetime,holiday,workingday,temp,atemp,humidity,windspeed,casual,registered,count,season_1,season_2,season_3,season_4,weather_1,weather_2,weather_3,weather_4,hour,month,day,year
01-01-2011 00:00,0,0,9.84,14.395,81,0.0,3,13,16,1,0,0,0,1,0,0,0,0,1,1,2011
01-01-2011 01:00,0,0,9.02,13.635,80,0.0,8,32,40,1,0,0,0,1,0,0,0,1,1,1,2011
01-01-2011 02:00,0,0,9.02,13.635,80,0.0,5,27,32,1,0,0,0,1,0,0,0,2,1,1,2011
01-01-2011 03:00,0,0,9.84,14.395,75,0.0,3,10,13,1,0,0,0,1,0,0,0,3,1,1,2011
01-01-2011 04:00,0,0,9.84,14.395,75,0.0,0,1,1,1,0,0,0,1,0,0,0,4,1,1,2011
01-01-2011 05:00,0,0,9.84,12.88,75,6.0032,0,1,1,1,0,0,0,0,1,0,0,5,1,1,2011
01-01-2011 06:00,0,0,9.02,13.635,80,0.0,2,0,2,1,0,0,0,1,0,0,0,6,1,1,2011
01-01-2011 07:00,0,0,8.2,12.88,86,0.0,1,2,3,1,0,0,0,1,0,0,0,7,1,1,2011
01-01-2011 08:00,0,0,9.84,14.395,75,0.0,1,7,8,1,0,0,0,1,0,0,0,8,1,1,2011
01-01-2011 09:00,0,0,13.12,17.425,76,0.0,8,6,14,1,0,0,0,1,0,0,0,9,1,1,2011


In [18]:

bs_df_encoded.printSchema()
bs_df_encoded = bs_df_encoded.drop('datetime')
bs_df_encoded = bs_df_encoded.withColumnRenamed("count", "label")

root
-- datetime: string (nullable = true)
-- holiday: integer (nullable = true)
-- workingday: integer (nullable = true)
-- temp: double (nullable = true)
-- atemp: double (nullable = true)
-- humidity: integer (nullable = true)
-- windspeed: double (nullable = true)
-- casual: integer (nullable = true)
-- registered: integer (nullable = true)
-- count: integer (nullable = true)
-- season_1: integer (nullable = true)
-- season_2: integer (nullable = true)
-- season_3: integer (nullable = true)
-- season_4: integer (nullable = true)
-- weather_1: integer (nullable = true)
-- weather_2: integer (nullable = true)
-- weather_3: integer (nullable = true)
-- weather_4: integer (nullable = true)
-- hour: integer (nullable = true)
-- month: integer (nullable = true)
-- day: integer (nullable = true)
-- year: integer (nullable = true)

In [19]:
#Explore how count varies with hour
import seaborn as sns
sns.set(style="ticks")
viz = bs_df_encoded.select('label', 'hour').distinct().limit(50)

display(viz)
#we can see from below which hours have more density

label,hour
61,13
179,8
79,10
154,13
77,23
15,14
155,14
250,20
92,8
376,15


In [20]:
#Explore how count varies with different features such as hour,month,etc#
import seaborn as sns
sns.set(style="ticks")
viz = bs_df_encoded.select('label', 'day').distinct().limit(50)
#sns.pairplot(viz, hue="species")
#plt.show()
display(viz)

label,day
92,2
203,4
71,4
214,7
43,7
163,7
179,8
137,11
83,11
307,2


In [21]:
#Split the dataset into train and train_test
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit
train, test = bs_df_encoded.randomSplit([0.9, 0.1], seed=12345)

In [22]:
#The features are assembled to send it to model
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(
    inputCols=["holiday","workingday","temp","atemp","humidity","windspeed","casual","registered","label","season_1","season_2","season_3","season_4","weather_1","weather_2","weather_3","weather_4", "hour", "month", "day", "year"],
    outputCol="features")

output = assembler.transform(train)
print("Assembled columns 'hour', 'day' etc  to vector column 'features'")
output.show(truncate=False)
print(output.count())
train_output = output.na.drop()
print(train_output.count())

Assembled columns 'hour', 'day' etc to vector column 'features'
+-------+----------+----+-----+--------+---------+------+----------+-----+--------+--------+--------+--------+---------+---------+---------+---------+----+-----+---+----+---------------------------------------------------------------------------------------------------+
holiday|workingday|temp|atemp|humidity|windspeed|casual|registered|label|season_1|season_2|season_3|season_4|weather_1|weather_2|weather_3|weather_4|hour|month|day|year|features |
+-------+----------+----+-----+--------+---------+------+----------+-----+--------+--------+--------+--------+---------+---------+---------+---------+----+-----+---+----+---------------------------------------------------------------------------------------------------+
0 |0 |3.28|2.275|79 |31.0009 |0 |24 |24 |1 |0 |0 |0 |0 |0 |1 |0 |1 |12 |2 |2012|(21,[2,3,4,5,7,8,9,15,17,18,19,20],[3.28,2.275,79.0,31.0009,24.0,24.0,1.0,1.0,1.0,12.0,2.0,2012.0])|
0 |0 |3.28|3.79 |53 |16.9979 |0 |26 |26 |1 |0 |0 |0 |1 |0 |0 |0 |8 |12 |2 |2012|(21,[2,3,4,5,7,8,9,13,17,18,19,20],[3.28,3.79,53.0,16.9979,26.0,26.0,1.0,1.0,8.0,12.0,2.0,2012.0]) |
0 |0 |3.28|4.545|53 |12.998 |0 |1 |1 |1 |0 |0 |0 |1 |0 |0 |0 |4 |9 |1 |2011|(21,[2,3,4,5,7,8,9,13,17,18,19,20],[3.28,4.545,53.0,12.998,1.0,1.0,1.0,1.0,4.0,9.0,1.0,2011.0]) |
0 |0 |3.28|4.545|53 |12.998 |0 |1 |1 |1 |0 |0 |0 |1 |0 |0 |0 |5 |9 |1 |2011|(21,[2,3,4,5,7,8,9,13,17,18,19,20],[3.28,4.545,53.0,12.998,1.0,1.0,1.0,1.0,5.0,9.0,1.0,2011.0]) |
0 |0 |3.28|4.545|53 |12.998 |1 |5 |6 |1 |0 |0 |0 |1 |0 |0 |0 |7 |9 |1 |2011|[0.0,0.0,3.28,4.545,53.0,12.998,1.0,5.0,6.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,7.0,9.0,1.0,2011.0] |
0 |0 |4.1 |2.275|46 |46.0022 |0 |14 |14 |1 |0 |0 |0 |0 |1 |0 |0 |3 |12 |2 |2012|(21,[2,3,4,5,7,8,9,14,17,18,19,20],[4.1,2.275,46.0,46.0022,14.0,14.0,1.0,1.0,3.0,12.0,2.0,2012.0]) |
0 |0 |4.1 |2.275|46 |47.9988 |0 |1 |1 |1 |0 |0 |0 |0 |1 |0 |0 |4 |12 |2 |2012|(21,[2,3,4,5,7,8,9,14,17,18,19,20],[4.1,2.275,46.0,47.9988,1.0,1.0,1.0,1.0,4.0,12.0,2.0,2012.0]) |
0 |0 |4.1 |3.03 |42 |31.0009 |0 |12 |12 |1 |0 |0 |0 |1 |0 |0 |0 |1 |9 |1 |2011|(21,[2,3,4,5,7,8,9,13,17,18,19,20],[4.1,3.03,42.0,31.0009,12.0,12.0,1.0,1.0,1.0,9.0,1.0,2011.0]) |
0 |0 |4.1 |3.03 |46 |31.0009 |0 |11 |11 |1 |0 |0 |0 |1 |0 |0 |0 |2 |9 |1 |2011|(21,[2,3,4,5,7,8,9,13,17,18,19,20],[4.1,3.03,46.0,31.0009,11.0,11.0,1.0,1.0,2.0,9.0,1.0,2011.0]) |
0 |0 |4.1 |3.03 |58 |39.0007 |1 |26 |27 |1 |0 |0 |0 |0 |1 |0 |0 |2 |12 |2 |2012|[0.0,0.0,4.1,3.03,58.0,39.0007,1.0,26.0,27.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,12.0,2.0,2012.0] |
0 |0 |4.1 |3.79 |42 |26.0027 |1 |24 |25 |1 |0 |0 |0 |1 |0 |0 |0 |0 |9 |1 |2011|(21,[2,3,4,5,6,7,8,9,13,18,19,20],[4.1,3.79,42.0,26.0027,1.0,24.0,25.0,1.0,1.0,9.0,1.0,2011.0]) |
0 |0 |4.1 |3.79 |46 |27.9993 |0 |4 |4 |1 |0 |0 |0 |1 |0 |0 |0 |3 |9 |1 |2011|(21,[2,3,4,5,7,8,9,13,17,18,19,20],[4.1,3.79,46.0,27.9993,4.0,4.0,1.0,1.0,3.0,9.0,1.0,2011.0]) |
0 |0 |4.1 |3.79 |49 |26.0027 |0 |2 |2 |1 |0 |0 |0 |1 |0 |0 |0 |6 |12 |2 |2012|(21,[2,3,4,5,7,8,9,13,17,18,19,20],[4.1,3.79,49.0,26.0027,2.0,2.0,1.0,1.0,6.0,12.0,2.0,2012.0]) |
0 |0 |4.1 |3.79 |49 |26.0027 |0 |3 |3 |1 |0 |0 |0 |1 |0 |0 |0 |5 |12 |2 |2012|(21,[2,3,4,5,7,8,9,13,17,18,19,20],[4.1,3.79,49.0,26.0027,3.0,3.0,1.0,1.0,5.0,12.0,2.0,2012.0]) |
0 |0 |4.1 |3.79 |68 |26.0027 |0 |21 |21 |1 |0 |0 |0 |0 |0 |1 |0 |0 |12 |2 |2012|(21,[2,3,4,5,7,8,9,15,18,19,20],[4.1,3.79,68.0,26.0027,21.0,21.0,1.0,1.0,12.0,2.0,2012.0]) |
0 |0 |4.1 |4.545|49 |19.0012 |0 |1 |1 |1 |0 |0 |0 |1 |0 |0 |0 |6 |9 |1 |2011|(21,[2,3,4,5,7,8,9,13,17,18,19,20],[4.1,4.545,49.0,19.0012,1.0,1.0,1.0,1.0,6.0,9.0,1.0,2011.0]) |
0 |0 |4.1 |4.545|49 |19.0012 |0 |10 |10 |1 |0 |0 |0 |1 |0 |0 |0 |8 |9 |1 |2011|(21,[2,3,4,5,7,8,9,13,17,18,19,20],[4.1,4.545,49.0,19.0012,10.0,10.0,1.0,1.0,8.0,9.0,1.0,2011.0]) |
0 |0 |4.1 |4.545|49 |19.9995 |3 |60 |63 |1 |0 |0 |0 |1 |0 |0 |0 |9 |12 |2 |2012|[0.0,0.0,4.1,4.545,49.0,19.9995,3.0,60.0,63.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,9.0,12.0,2.0,2012.0] |
0 |0 |4.92|3.79 |46 |35.0

In [23]:
test_output = assembler.transform(test)
print(test_output.count())
train_output = test_output.na.drop()
print(test_output.count())
print("Assembled columns 'hour', 'day' etc  to vector column 'features'")
test_output.show(truncate=False)#.select("features", "clicked")

1089
1089
Assembled columns 'hour', 'day' etc to vector column 'features'
+-------+----------+----+------+--------+---------+------+----------+-----+--------+--------+--------+--------+---------+---------+---------+---------+----+-----+---+----+------------------------------------------------------------------------------------------------------+
holiday|workingday|temp|atemp |humidity|windspeed|casual|registered|label|season_1|season_2|season_3|season_4|weather_1|weather_2|weather_3|weather_4|hour|month|day|year|features |
+-------+----------+----+------+--------+---------+------+----------+-----+--------+--------+--------+--------+---------+---------+---------+---------+----+-----+---+----+------------------------------------------------------------------------------------------------------+
0 |0 |3.28|4.545 |53 |12.998 |0 |18 |18 |1 |0 |0 |0 |1 |0 |0 |0 |7 |12 |2 |2012|(21,[2,3,4,5,7,8,9,13,17,18,19,20],[3.28,4.545,53.0,12.998,18.0,18.0,1.0,1.0,7.0,12.0,2.0,2012.0]) |
0 |0 |4.1 |3.03 |39 |30.0026 |0 |22 |22 |1 |0 |0 |0 |1 |0 |0 |0 |23 |8 |1 |2011|(21,[2,3,4,5,7,8,9,13,17,18,19,20],[4.1,3.03,39.0,30.0026,22.0,22.0,1.0,1.0,23.0,8.0,1.0,2011.0]) |
0 |0 |5.74|7.575 |43 |11.0014 |0 |28 |28 |1 |0 |0 |0 |1 |0 |0 |0 |22 |12 |2 |2012|(21,[2,3,4,5,7,8,9,13,17,18,19,20],[5.74,7.575,43.0,11.0014,28.0,28.0,1.0,1.0,22.0,12.0,2.0,2012.0]) |
0 |0 |6.56|6.06 |40 |31.0009 |4 |92 |96 |1 |0 |0 |0 |1 |0 |0 |0 |18 |12 |2 |2012|[0.0,0.0,6.56,6.06,40.0,31.0009,4.0,92.0,96.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,18.0,12.0,2.0,2012.0] |
0 |0 |6.56|6.82 |40 |22.0028 |4 |44 |48 |1 |0 |0 |0 |1 |0 |0 |0 |18 |9 |1 |2011|[0.0,0.0,6.56,6.82,40.0,22.0028,4.0,44.0,48.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,18.0,9.0,1.0,2011.0] |
0 |0 |6.56|6.82 |47 |19.0012 |5 |38 |43 |1 |0 |0 |0 |0 |1 |0 |0 |2 |15 |1 |2012|[0.0,0.0,6.56,6.82,47.0,19.0012,5.0,38.0,43.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,15.0,1.0,2012.0] |
0 |0 |6.56|6.82 |48 |26.0027 |1 |24 |25 |1 |0 |0 |0 |0 |1 |0 |0 |3 |15 |1 |2012|[0.0,0.0,6.56,6.82,48.0,26.0027,1.0,24.0,25.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,3.0,15.0,1.0,2012.0] |
0 |0 |6.56|9.85 |59 |6.0032 |2 |18 |20 |1 |0 |0 |0 |0 |1 |0 |0 |1 |15 |1 |2011|[0.0,0.0,6.56,9.85,59.0,6.0032,2.0,18.0,20.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,15.0,1.0,2011.0] |
0 |0 |6.56|9.85 |69 |6.0032 |3 |27 |30 |1 |0 |0 |0 |1 |0 |0 |0 |0 |12 |2 |2011|(21,[2,3,4,5,6,7,8,9,13,18,19,20],[6.56,9.85,69.0,6.0032,3.0,27.0,30.0,1.0,1.0,12.0,2.0,2011.0]) |
0 |0 |6.56|11.365|59 |0.0 |0 |1 |1 |1 |0 |0 |0 |1 |0 |0 |0 |5 |15 |1 |2011|(21,[2,3,4,7,8,9,13,17,18,19,20],[6.56,11.365,59.0,1.0,1.0,1.0,1.0,5.0,15.0,1.0,2011.0]) |
0 |0 |7.38|12.12 |55 |0.0 |1 |30 |31 |0 |0 |0 |1 |1 |0 |0 |0 |7 |11 |12 |2011|(21,[2,3,4,6,7,8,12,13,17,18,19,20],[7.38,12.12,55.0,1.0,30.0,31.0,1.0,1.0,7.0,11.0,12.0,2011.0]) |
0 |0 |8.2 |10.605|59 |11.0014 |1 |11 |12 |0 |0 |0 |1 |1 |0 |0 |0 |7 |18 |12 |2011|[0.0,0.0,8.2,10.605,59.0,11.0014,1.0,11.0,12.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,7.0,18.0,12.0,2011.0] |
0 |0 |8.2 |10.605|69 |8.9981 |2 |3 |5 |1 |0 |0 |0 |0 |1 |0 |0 |6 |13 |2 |2011|[0.0,0.0,8.2,10.605,69.0,8.9981,2.0,3.0,5.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,6.0,13.0,2.0,2011.0] |
0 |0 |8.2 |10.605|75 |11.0014 |7 |73 |80 |0 |0 |0 |1 |1 |0 |0 |0 |22 |17 |12 |2011|[0.0,0.0,8.2,10.605,75.0,11.0014,7.0,73.0,80.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,22.0,17.0,12.0,2011.0] |
0 |0 |8.2 |12.88 |59 |0.0 |19 |88 |107 |0 |0 |0 |1 |1 |0 |0 |0 |9 |11 |12 |2011|(21,[2,3,4,6,7,8,12,13,17,18,19,20],[8.2,12.88,59.0,19.0,88.0,107.0,1.0,1.0,9.0,11.0,12.0,2011.0]) |
0 |0 |8.2 |12.88 |69 |0.0 |1 |31 |32 |0 |0 |0 |1 |1 |0 |0 |0 |8 |18 |12 |2011|(21,[2,3,4,6,7,8,12,13,17,18,19,20],[8.2,12.88,69.0,1.0,31.0,32.0,1.0,1.0,8.0,18.0,12.0,2011.0]) |
0 |0 |9.02|10.605|55 |19.0012 |0 |18 |18 |1 |0 |0 |0 |1 |0 |0 |0 |8 |16 |1 |2011|(21,[2,3,4,5,7,8,9,13,17,18,19,20],[9.02,10.605,55.0,19.0012,18.0,18.0,1.0,1.0,8.0,16.0,1.0,2011.0]) |
0 |0 |9.02|11.365|41 |8.9981 |7 |88 |95 |0 |0 |0 |1 |1 |0 |0 |0 |22 |10 |12 |2011|[0.0,0.0,9.02,11.365,41.0,8.9981,7.0,88.0,95.0,0.

In [24]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.regression import LinearRegression
lr = LinearRegression(maxIter=10)

# Fit the model
lrModel = lr.fit(train_output)

In [25]:
# Print the coefficients and intercept for logistic regression
print("Coefficients: " + str(lrModel.coefficients))
print("Intercept: " + str(lrModel.intercept))

Coefficients: [0.241007000329,0.0240300559307,-0.00329772606512,-0.0038201221511,-0.00311898556861,0.00062578456448,0.563257347413,0.563189130437,0.436769734736,0.647037334628,0.239546459467,-0.170680495115,-0.686157844747,-0.0505300374949,0.0248775702221,0.0845720083375,0.0,0.00476996706435,0.00572325553762,0.158668761597,-0.0867775035524]
Intercept: 173.7435412550812

In [26]:
import pyspark.sql.functions
predictions = lrModel.transform(test_output)\
    .select("features", "label", "prediction")\
    .show(1000)

from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.mllib.evaluation import BinaryClassificationMetrics
# testRDD = test.rdd 
# predictionAndLabels = testRDD.map(lambda lp: (float(model.predict(lp.features)), lp.label))
# # Evaluate model
# metrics = BinaryClassificationMetrics(predictionAndLabels)
# f1Score = metrics.fMeasure()
# print(f1Score)
from pyspark.ml.evaluation import RegressionEvaluator
lr_evaluator = RegressionEvaluator(predictionCol="prediction", labelCol="label",metricName="r2")
# print("R Squared (R2) on test data = %g" % lr_evaluator.evaluate(predictions))


+--------------------+-----+------------------+
 features|label| prediction|
+--------------------+-----+------------------+
(21,[2,3,4,5,7,8,...| 18|17.977026052947167|
(21,[2,3,4,5,7,8,...| 22|22.015787070326525|
(21,[2,3,4,5,7,8,...| 28|28.058417248633106|
[0.0,0.0,6.56,6.0...| 96| 96.06176876485841|
[0.0,0.0,6.56,6.8...| 48| 47.96614804695298|
[0.0,0.0,6.56,6.8...| 43| 42.88936084849334|
[0.0,0.0,6.56,6.8...| 25| 24.89586081959351|
[0.0,0.0,6.56,9.8...| 20| 19.91497309035597|
(21,[2,3,4,5,6,7,...| 30|29.944761523582343|
(21,[2,3,4,7,8,9,...| 1|0.8497462836939462|
(21,[2,3,4,6,7,8,...| 31|31.254276180719643|
[0.0,0.0,8.2,10.6...| 12|12.292612444907064|
[0.0,0.0,8.2,10.6...| 5| 5.049054042133719|
[0.0,0.0,8.2,10.6...| 80| 80.30614705988239|
(21,[2,3,4,6,7,8,...| 107|107.24383440310577|
(21,[2,3,4,6,7,8,...| 32|32.249794575552045|
(21,[2,3,4,5,7,8,...| 18| 17.88823763502819|
[0.0,0.0,9.02,11....| 95| 95.36465169562645|
[0.0,0.0,9.02,11....| 8| 8.135853418432134|
[0.0,0.0,9.02,11....| 52| 51.98915229067231|
[0.0,0.0,9.02,12....| 72| 72.31418633732791|
[0.0,0.0,9.02,12....| 4| 4.025966815793339|
(21,[2,3,4,5,7,8,...| 4|3.9795123747819616|
[0.0,0.0,9.84,11....| 99| 98.96947847483119|
[0.0,0.0,9.84,11....| 129| 129.2948408412329|
(21,[2,3,4,5,7,8,...| 14|14.113672879200266|
[0.0,0.0,9.84,11....| 18| 18.10894302375641|
[0.0,0.0,9.84,12....| 61| 61.05115486280475|
[0.0,0.0,9.84,12....| 46| 46.11333010705363|
[0.0,0.0,9.84,12....| 9| 9.141253567713619|
[0.0,0.0,9.84,12....| 17|16.955243127583742|
(21,[2,3,4,5,7,8,...| 5| 4.95616342359574|
[0.0,0.0,9.84,12....| 28|27.847185391779135|
[0.0,0.0,9.84,12....| 78| 77.95309336498097|
[0.0,0.0,9.84,12....| 45| 45.10487155512979|
[0.0,0.0,9.84,12....| 4|3.7771615441562574|
[0.0,0.0,9.84,12....| 6| 6.146075969984423|
[0.0,0.0,10.66,12...| 15|15.144829784253773|
[0.0,0.0,10.66,12...| 172|172.34575491300694|
(21,[2,3,4,5,7,8,...| 1|0.9438661799450188|
(21,[2,3,4,5,7,8,...| 1|0.9486361470093811|
[0.0,0.0,10.66,12...| 50| 50.14637929148981|
[0.0,0.0,10.66,12...| 49| 48.98228096317945|
[0.0,0.0,10.66,12...| 168|168.00876012888256|
[0.0,0.0,10.66,12...| 66| 66.2409630697886|
(21,[2,3,4,5,7,8,...| 3| 3.267321738264741|
(21,[2,3,4,5,6,7,...| 86| 85.77989784022225|
[0.0,0.0,10.66,13...| 71| 70.89031795353225|
[0.0,0.0,10.66,13...| 40| 39.57734920046383|
[0.0,0.0,10.66,14...| 93| 93.06009491849623|
(21,[2,3,4,7,8,12...| 3|3.1725717080915956|
(21,[2,3,4,6,7,8,...| 50|50.057693630037136|
(21,[2,3,4,6,7,8,...| 108|107.97076957010259|
(21,[2,3,4,6,7,8,...| 11|11.078595354463971|
[0.0,0.0,11.48,12...| 64| 64.1081916365985|
[0.0,0.0,11.48,12...| 237|237.34529434147143|
[0.0,0.0,11.48,12...| 84| 84.0582884154235|
[0.0,0.0,11.48,13...| 172|172.06664761252185|
(21,[2,3,4,5,7,8,...| 5| 4.590642629069123|
[0.0,0.0,11.48,14...| 45| 44.9223066059663|
(21,[2,3,4,6,7,8,...| 41| 41.10475282609491|
[0.0,0.0,12.3,12....| 44|44.203386590063644|
[0.0,0.0,12.3,14....| 84| 84.00857143095209|
[0.0,0.0,12.3,14....| 219|219.32691227803898|
[0.0,0.0,12.3,14....| 58| 58.21971609892992|
[0.0,0.0,12.3,14....| 272|271.96960672279795|
[0.0,0.0,12.3,14....| 302|301.90714526562067|
[0.0,0.0,12.3,14....| 12|12.013101356340968|
(21,[2,3,4,5,7,8,...| 13| 12.83519063766613|
(21,[2,3,4,5,6,7,...| 53| 52.88630185280664|
(21,[2,3,4,5,7,8,...| 12|11.878026877707214|
[0.0,0.0,12.3,15....| 109|109.30208135575764|
(21,[2,3,4,5,6,7,...| 83| 83.10680608899023|
[0.0,0.0,12.3,15....| 170| 170.0592824598628|
(21,[2,3,4,6,7,8,...| 24| 23.78963295315353|
(21,[2,3,4,6,7,8,...| 74| 74.0324517635435|
(21,[2,3,4,6,7,8,...| 158|158.11533511328275|
[0.0,0.0,13.12,14...| 30|29.879588673145747|
[0.0,0.0,13.12,14...| 236| 236.2582732593253|
[0.0,0.0,13.12,15...| 357|357.15064505412096|
[0.0,0.0,13.12,15...| 74| 74.02681011512091|
[0.0,0.0,13.12,15...| 212|212.08742202368924|
[0.0,0.0,13.12,15...| 117|116.98839861707657|
[0.0,0.0,13.12,15...| 77| 76.88699661194131|
[0.0,0.0,13.12,15...| 55| 54.89386773038325|
[0.0,0.0,13.12,15...| 204| 203.9879390504792|
[0.0,0.

In [27]:
# Parameter grid search for best parameters to give good predictions
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.regression import LinearRegression
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit
# We use a ParamGridBuilder to construct a grid of parameters to search over.
# TrainValidationSplit will try all combinations of values and determine best model using
# the evaluator.
paramGrid = ParamGridBuilder()\
    .addGrid(lr.regParam, [0.1, 0.01]) \
    .addGrid(lr.fitIntercept, [False, True])\
    .addGrid(lr.elasticNetParam, [0.0, 0.5, 1.0])\
    .build()

# In this case the estimator is simply the linear regression.
# A TrainValidationSplit requires an Estimator, a set of Estimator ParamMaps, and an Evaluator.
tvs = TrainValidationSplit(estimator=lr,
                           estimatorParamMaps=paramGrid,
                           evaluator=RegressionEvaluator(),
                           # 80% of the data will be used for training, 20% for validation.
                           trainRatio=0.8)

# Run TrainValidationSplit, and choose the best set of parameters.
model = tvs.fit(train_output)

# Make predictions on test data. model is the model with combination of parameters
# that performed best.
model.transform(test_output)\
    .select("features", "label", "prediction")\
    .show()

+--------------------+-----+------------------+
 features|label| prediction|
+--------------------+-----+------------------+
(21,[2,3,4,5,7,8,...| 18| 17.99775672379881|
(21,[2,3,4,5,7,8,...| 22|22.002205520528005|
(21,[2,3,4,5,7,8,...| 28|28.002288892258296|
[0.0,0.0,6.56,6.0...| 96| 95.99923333047171|
[0.0,0.0,6.56,6.8...| 48| 48.00084264442458|
[0.0,0.0,6.56,6.8...| 43| 42.99742220253684|
[0.0,0.0,6.56,6.8...| 25|24.998372144993553|
[0.0,0.0,6.56,9.8...| 20|19.997464039007003|
(21,[2,3,4,5,6,7,...| 30|29.996023420750365|
(21,[2,3,4,7,8,9,...| 1|0.9985058350368861|
(21,[2,3,4,6,7,8,...| 31|31.001512686145304|
[0.0,0.0,8.2,10.6...| 12|12.002093905310927|
[0.0,0.0,8.2,10.6...| 5| 4.999609964559781|
[0.0,0.0,8.2,10.6...| 80| 80.002062511681|
(21,[2,3,4,6,7,8,...| 107|106.99961782613572|
(21,[2,3,4,6,7,8,...| 32| 32.00095997646687|
(21,[2,3,4,5,7,8,...| 18|17.999483141037537|
[0.0,0.0,9.02,11....| 95| 95.00380867805801|
[0.0,0.0,9.02,11....| 8| 8.00120250683875|
[0.0,0.0,9.02,11....| 52| 51.9964103597502|
+--------------------+-----+------------------+
only showing top 20 rows

In [28]:
# Random Forest Classifier model
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.evaluation import RegressionEvaluator
rf = RandomForestRegressor(labelCol="label", featuresCol="features", numTrees=100)
# Train model.  This also runs the indexers.
rf_model = rf.fit(train_output)
# rf_model.persist()
# Make predictions.
predictions = rf_model.transform(test_output)

# Select example rows to display.
predictions.select("prediction", "label", "features").show(5)

# Select (prediction, true label) and compute test error
evaluator = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

+------------------+-----+--------------------+
 prediction|label| features|
+------------------+-----+--------------------+
 30.31997830000496| 18|(21,[2,3,4,5,7,8,...|
 34.87887391863016| 22|(21,[2,3,4,5,7,8,...|
 40.25137682428468| 28|(21,[2,3,4,5,7,8,...|
 93.14719788818678| 96|[0.0,0.0,6.56,6.0...|
55.045113587081005| 48|[0.0,0.0,6.56,6.8...|
+------------------+-----+--------------------+
only showing top 5 rows

Root Mean Squared Error (RMSE) on test data = 16.9318

In [29]:
# GBT Regressor model 
from pyspark.ml.regression import GBTRegressor
gbt = GBTRegressor(featuresCol="features", maxIter=10)

gbt_model = gbt.fit(train_output)
# Make predictions.
predictions = gbt_model.transform(test_output)


gbt_model.write().overwrite().save("bike_sharing_gbt.model")
# Select example rows to display.
predictions.select("prediction", "label", "features").show(5)

# Select (prediction, true label) and compute test error
evaluator = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)
#Gave root mean square error 


+------------------+-----+--------------------+
 prediction|label| features|
+------------------+-----+--------------------+
16.889233655915504| 18|(21,[2,3,4,5,7,8,...|
 16.92511614166162| 22|(21,[2,3,4,5,7,8,...|
 30.32523546505164| 28|(21,[2,3,4,5,7,8,...|
 95.83936857912708| 96|[0.0,0.0,6.56,6.0...|
46.025330660611566| 48|[0.0,0.0,6.56,6.8...|
+------------------+-----+--------------------+
only showing top 5 rows

Root Mean Squared Error (RMSE) on test data = 7.24449